In [1]:
#Processing evcc block ack
import csv
import numpy as np

with open('csv/DataLink/max_interf/evcc_blockack_max.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    startint_seq_num_block_ack = []
    timelist_block_ack = []
    
    for row in rows:
        startint_seq_num_block_ack.append(int(row['Starting Sequence Number']))
        timelist_block_ack.append(float(row['Time']))

In [2]:
#for debug
if(len(startint_seq_num_block_ack) != len(timelist_block_ack)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_block_ack))

986


In [3]:
#Processing evcc send
with open('csv/DataLink/max_interf/evcc_send_max.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_evcc_send = []
    seq_num_evcc_send = []
    data_length_evcc_send = []
    info_evcc_send = []
    frame_num_evcc_send = []
    
    for row in rows:
        timelist_evcc_send.append(float(row['Time']))
        seq_num_evcc_send.append(int(row['Sequence number']))
        data_length_evcc_send.append(int(row['Data Length']))
        info_evcc_send.append(str(row['Info']))
        frame_num_evcc_send.append(int(row['No.']))

In [4]:
#for debug
if(len(timelist_evcc_send) != len(seq_num_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(data_length_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(info_evcc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send) != len(frame_num_evcc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send))

1517


In [5]:
timelist_evcc_send_split = []
seq_num_evcc_send_split = []
data_length_evcc_send_split = []
info_evcc_send_split = []
frame_num_evcc_send_split = []

round_start_point = 0
round_end_point = 0

sdpreq_index_list = []
#Fetch first packet index of data length = 74 (SDP request) in each round
for i in range(0, len(data_length_evcc_send)):
    if(i == 0 and data_length_evcc_send[i] == 74):
        sdpreq_index_list.append(i)
    if(i > 0 and data_length_evcc_send[i] == 74 and data_length_evcc_send[i-1] != 74):
        sdpreq_index_list.append(i)
sdpreq_index_list.append(len(data_length_evcc_send)) #Add the end of list index

for i in range(0, len(sdpreq_index_list)-1):
    timelist_evcc_send_split.append([timelist_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    seq_num_evcc_send_split.append([seq_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    data_length_evcc_send_split.append([data_length_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    info_evcc_send_split.append([info_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])
    frame_num_evcc_send_split.append([frame_num_evcc_send[j] for j in range(sdpreq_index_list[i], sdpreq_index_list[i+1])])

In [6]:
#for debug
if(len(timelist_evcc_send_split) != len(seq_num_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(data_length_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(info_evcc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_evcc_send_split) != len(frame_num_evcc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_evcc_send_split)) #Output # of testing rounds

20


In [7]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_evcc_send_split)):
    print(timelist_evcc_send_split[i][0]-timelist_evcc_send_split[i-1][0])

21.419687
21.973245999999996
22.919047000000006
21.173198999999997
22.798350000000013
22.320474999999988
21.305890000000005
22.321897000000007
22.170615999999995
22.234062999999992
22.069951999999972
21.284245000000055
21.60061399999995
20.104599000000007
21.995933000000036
21.59201299999995
22.530367000000012
21.47176300000001
21.181276000000025


In [8]:
#Set evcc time anchor in each round per stage
evcc_send_time_anchor = []
#data_length_in_each_stage = [74, 96, 189, 173, 173, 173, 185, 461, 189, 173]
wierd_packet = {}
packet_loss_testing_round_5_flag = False
packet_loss_testing_round_13_flag = False
packet_loss_testing_round_13_1_flag = False
packet_loss_testing_round_13_2_flag = False
packet_loss_testing_round_13_3_flag = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(evcc_send_time_anchor_each_round):
    print("Time difference: " + str(evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-1]-
                          evcc_send_time_anchor_each_round[len(evcc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        print("\nFalse")
        print("Stage Num = " + str(stageNum))
        print("Valid Stage Num = " + str(valid_stageNum))
        print("What is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet.get(frameNum) != None):
        del wierd_packet[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_evcc_send_split)):
#for i in [2]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    evcc_send_time_anchor_each_round = [timelist_evcc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_evcc_send_split[i][0], info_evcc_send_split[i][0])
    stageNum = stageNum + 1
    
    for j in range(1, len(data_length_evcc_send_split[i])):

        #Deal with packet loss......
        if(i == 5 and packet_loss_testing_round_5_flag == False):
            evcc_send_time_anchor_each_round.append(135.6)
            printinfo(stageNum, 679483, "Packet Loss")
            print_time_difference(evcc_send_time_anchor_each_round)
            stageNum = stageNum + 1
            packet_loss_testing_round_5_flag = True
        
        #Stage 2
        if(data_length_evcc_send_split[i][j] == 96 and data_length_evcc_send_split[i][j-1] != 96):
            if(checkstage_num(stageNum, 2, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 3
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 3, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 4 
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            #if(info_evcc_send_split[i][j][-2:] == 'TC'):
            if(checkstage_num(stageNum, 4, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                continue
           
        #Stage 5, tricky
        if(data_length_evcc_send_split[i][j] == 173):    
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
            
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                if(i == 1):
                    for k in range(1, 4):
                        if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                            isTCfound = True
                            break
                else:
                    for k in range(1, 3):
                        if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                            isTCfound = True
                            break
                
                if(not isTCfound):
                    if(checkstage_num(stageNum, 5, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1  
                        
                        #Deal with packet loss......
                        if(i == 13 and packet_loss_testing_round_13_flag == False):
                            evcc_send_time_anchor_each_round.append(311.399807)
                            printinfo(stageNum, 1652776, "Packet Loss")
                            print_time_difference(evcc_send_time_anchor_each_round)
                            stageNum = stageNum + 1
                            packet_loss_testing_round_13_flag = True
                                 
                        continue
                
        #Stage 6, tricky
        if(data_length_evcc_send_split[i][j] == 173):
            
            #Avoid TCP retransmission packet
            if(i != 2 or (i == 2 and seq_num_evcc_send_split[i][j] == 2263)):
            
                if(info_evcc_send_split[i][j][-2:] == 'TC'):
                    if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                          timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                          seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1

                        continue

                else: #info_evcc_send_split[i][j][-2:] == 'FC'
                    #Backward checking 2 packets
                    isTCfound = False
                    for k in range(1, 3):
                        if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                            isTCfound = True
                            break

                    if(not isTCfound):
                        if(checkstage_num(stageNum, 6, info_evcc_send_split[i][j],
                                          timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                          seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                            print("Stage " + str(stageNum) +
                                ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                                ", Data length " + str(data_length_evcc_send_split[i][j]) +
                                ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                            evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                            printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                            print_time_difference(evcc_send_time_anchor_each_round)
                            stageNum = stageNum + 1

                            continue

        #Stage 7
        if(data_length_evcc_send_split[i][j] == 185 and data_length_evcc_send_split[i][j-1] != 185):
            if(checkstage_num(stageNum, 7, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 13 and packet_loss_testing_round_13_1_flag == False):
                    evcc_send_time_anchor_each_round.append(311.701577)
                    printinfo(stageNum, 1654007, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_13_1_flag = True
                #Deal with packet loss......
                if(i == 13 and packet_loss_testing_round_13_2_flag == False):
                    evcc_send_time_anchor_each_round.append(311.837217)
                    printinfo(stageNum, 1654514, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_13_2_flag = True
                #Deal with packet loss......
                if(i == 13 and packet_loss_testing_round_13_3_flag == False):
                    evcc_send_time_anchor_each_round.append(311.886181)
                    printinfo(stageNum, 1654723, "Packet Loss")
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_13_3_flag = True
                
                continue
                
        #Stage 8
        if(data_length_evcc_send_split[i][j] == 461 and data_length_evcc_send_split[i][j-1] != 461):
            if(checkstage_num(stageNum, 8, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
        #Stage 9
        if(data_length_evcc_send_split[i][j] == 189 and data_length_evcc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 9, info_evcc_send_split[i][j],
                                  timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                  seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                print_time_difference(evcc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_evcc_send_split[i][j] == 173 and data_length_evcc_send_split[i][j-1] != 173):
            
            if(info_evcc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                    evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                    printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                    print_time_difference(evcc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
            else: #info_evcc_send_split[i][j][-2:] == 'FC'
                #Backward checking 2 packets
                isTCfound = False
                for k in range(1, 3):
                    if(data_length_evcc_send_split[i][j-k] == 173 and info_evcc_send_split[i][j-k][-2:] == 'TC'):
                        isTCfound = True
                        break

                if(not isTCfound):
                    if(checkstage_num(stageNum, 10, info_evcc_send_split[i][j],
                                      timelist_evcc_send_split[i][j], data_length_evcc_send_split[i][j],
                                      seq_num_evcc_send_split[i][j], frame_num_evcc_send_split[i][j])):
                        print("Stage " + str(stageNum) +
                            ", Not TC? No. " + str(frame_num_evcc_send_split[i][j]) +
                            ", Data length " + str(data_length_evcc_send_split[i][j]) +
                            ", Info " + str(info_evcc_send_split[i][j]) + " , Count FC because TC cannot be found.")
                        evcc_send_time_anchor_each_round.append(timelist_evcc_send_split[i][j])
                        printinfo(stageNum, frame_num_evcc_send_split[i][j], info_evcc_send_split[i][j])
                        print_time_difference(evcc_send_time_anchor_each_round)
                        stageNum = stageNum + 1
                
        if(stageNum == 11): break
        
    evcc_send_time_anchor.append(evcc_send_time_anchor_each_round)


-------- Test Round 1 --------
-----------------------------------------
Stage 1 finish. No. 92526, Info QoS Data, SN=1973, FN=0, Flags=.p..R..TC

False
Stage Num = 2
Valid Stage Num = 3
What is this packet? Data Length = 189 No. 95340
Time 26.322231 #Seq 1985 Info QoS Data, SN=1985, FN=0, Flags=.p.....TC


False
Stage Num = 2
Valid Stage Num = 9
What is this packet? Data Length = 189 No. 95340
Time 26.322231 #Seq 1985 Info QoS Data, SN=1985, FN=0, Flags=.p.....TC


False
Stage Num = 2
Valid Stage Num = 4
What is this packet? Data Length = 173 No. 96077
Time 26.550282 #Seq 1987 Info QoS Data, SN=1987, FN=0, Flags=.p.....TC


False
Stage Num = 2
Valid Stage Num = 5
What is this packet? Data Length = 173 No. 96077
Time 26.550282 #Seq 1987 Info QoS Data, SN=1987, FN=0, Flags=.p.....TC


False
Stage Num = 2
Valid Stage Num = 6
What is this packet? Data Length = 173 No. 96077
Time 26.550282 #Seq 1987 Info QoS Data, SN=1987, FN=0, Flags=.p.....TC


False
Stage Num = 2
Valid Stage Num = 10
W

In [9]:
#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print("i = " + str(i))
    print(len(evcc_send_time_anchor[i]))

20

i = 0
1
i = 1
10
i = 2
10
i = 3
10
i = 4
10
i = 5
10
i = 6
10
i = 7
10
i = 8
10
i = 9
10
i = 10
10
i = 11
10
i = 12
10
i = 13
10
i = 14
10
i = 15
8
i = 16
10
i = 17
10
i = 18
10
i = 19
10


In [10]:
#for debug
for i in range(0, len(evcc_send_time_anchor)):
    print("i = " + str(i))
    print(evcc_send_time_anchor[i][len(evcc_send_time_anchor[i])-1]-evcc_send_time_anchor[i][0])

i = 0
0.0
i = 1
2.825794000000002
i = 2
2.916784000000007
i = 3
2.2251839999999987
i = 4
3.265495999999999
i = 5
2.835628999999983
i = 6
1.9320160000000044
i = 7
3.304869999999994
i = 8
2.6081129999999746
i = 9
2.9662040000000047
i = 10
2.742227000000014
i = 11
2.0673830000000066
i = 12
2.2568409999999517
i = 13
1.2302150000000438
i = 14
2.2061140000000137
i = 15
1.937896999999964
i = 16
2.3920590000000175
i = 17
2.291643000000022
i = 18
2.025845000000004
i = 19
3.193886999999961


In [11]:
#Processing secc send
with open('csv/DataLink/max_interf/secc_send_max_add_1_round_SDP_res.csv', newline='') as csvfile:
    rows = csv.DictReader(csvfile)
    timelist_secc_send = []
    seq_num_secc_send = []
    data_length_secc_send = []
    info_secc_send = []
    frame_num_secc_send = []
    
    for row in rows:
        timelist_secc_send.append(float(row['Time']))
        seq_num_secc_send.append(int(row['Sequence number']))
        data_length_secc_send.append(int(row['Data Length']))
        info_secc_send.append(str(row['Info']))
        frame_num_secc_send.append(int(row['No.']))

In [12]:
#for debug
if(len(timelist_secc_send) != len(seq_num_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(data_length_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(info_secc_send)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send) != len(frame_num_secc_send)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send))

1070


In [13]:
timelist_secc_send_split = []
seq_num_secc_send_split = []
data_length_secc_send_split = []
info_secc_send_split = []
frame_num_secc_send_split = []

round_start_point = 0
round_end_point = 0

sdpres_index_list = []
#Fetch first packet index of data length = 92 (SDP response) in each round
for i in range(0, len(data_length_secc_send)):
    if(i == 0 and data_length_secc_send[i] == 92):
        sdpres_index_list.append(i)
    if(i > 0 and data_length_secc_send[i] == 92 and data_length_secc_send[i-1] != 92):
        sdpres_index_list.append(i)
sdpres_index_list.append(len(data_length_secc_send)) #Add the end of list index

for i in range(0, len(sdpres_index_list)-1):
        timelist_secc_send_split.append([timelist_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        seq_num_secc_send_split.append([seq_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        data_length_secc_send_split.append([data_length_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        info_secc_send_split.append([info_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])
        frame_num_secc_send_split.append([frame_num_secc_send[j] for j in range(sdpres_index_list[i], sdpres_index_list[i+1])])

In [14]:
#for debug
if(len(timelist_secc_send_split) != len(seq_num_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(data_length_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(info_secc_send_split)):
    print("Error. Length is not equal.")
elif(len(timelist_secc_send_split) != len(frame_num_secc_send_split)):
    print("Error. Length is not equal.")
else:
    print(len(timelist_secc_send_split)) #Output # of testing rounds

20


In [15]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i))
    print(len(timelist_secc_send_split[i]))

i = 0
36
i = 1
49
i = 2
57
i = 3
50
i = 4
55
i = 5
53
i = 6
57
i = 7
52
i = 8
56
i = 9
57
i = 10
59
i = 11
58
i = 12
54
i = 13
50
i = 14
57
i = 15
45
i = 16
56
i = 17
54
i = 18
56
i = 19
59


In [16]:
#for debug, this is ideally the time of each testing round
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

26.550628000000003
22.028153000000003
22.81058399999999
21.360501999999997
22.804158
22.130099
21.304294999999996
22.422740000000005
22.114607000000007
22.22235599999999
22.12243300000003
21.201747999999952
21.59241700000001
20.185448000000008
21.99704300000002
21.51047299999999
22.554220999999984
21.485689000000036
21.190412999999978


In [17]:
for i in range(0, len(timelist_secc_send_split)):
    print('i = ' + str(i) + ' ', end = '')
    print(len(timelist_secc_send_split[i]))

i = 0 36
i = 1 49
i = 2 57
i = 3 50
i = 4 55
i = 5 53
i = 6 57
i = 7 52
i = 8 56
i = 9 57
i = 10 59
i = 11 58
i = 12 54
i = 13 50
i = 14 57
i = 15 45
i = 16 56
i = 17 54
i = 18 56
i = 19 59


In [18]:
for i in range(1, len(timelist_secc_send_split)):
    print(timelist_secc_send_split[i][0]-timelist_secc_send_split[i-1][0])

26.550628000000003
22.028153000000003
22.81058399999999
21.360501999999997
22.804158
22.130099
21.304294999999996
22.422740000000005
22.114607000000007
22.22235599999999
22.12243300000003
21.201747999999952
21.59241700000001
20.185448000000008
21.99704300000002
21.51047299999999
22.554220999999984
21.485689000000036
21.190412999999978


In [19]:
#Set secc time anchor in each round per stage
secc_send_time_anchor = []
#data_length_in_each_stage = [92, 173, 157, 189, 253, 173, 189, 173, 493, 173]
wierd_packet_secc = {}
packet_loss_testing_round_1_flag = False
packet_loss_testing_round_4_flag = False
packet_loss_testing_round_7_flag = False

def printinfo(stageNum, frameNum, info_string):
    print("-----------------------------------------")
    print("Stage " + str(stageNum) + " finish. No. " + str(frameNum) + ", Info " + str(info_string))

def print_time_difference(secc_send_time_anchor_each_round):
    print("Time difference: " + str(secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-1]-
                          secc_send_time_anchor_each_round[len(secc_send_time_anchor_each_round)-2]))
    
def checkstage_num(stageNum, valid_stageNum, info_string, time_string, datalen, seqnum, frameNum):
    if(stageNum == valid_stageNum):
        check_wield_packet_list(frameNum)
        return True
    else:
        #debug
        print("\nWhat is this packet? Data Length = " + str(datalen) + " No. " + str(frameNum))
        print("Time " + str(time_string) + " #Seq " + str(seqnum) + " Info " + str(info_string) + "\n")
        wierd_packet_secc[frameNum] = [datalen, stageNum]
        return False

def check_wield_packet_list(frameNum):
    if(wierd_packet_secc.get(frameNum) != None):
        del wierd_packet_secc[frameNum]
        print("\nNo. " + str(frameNum) + " has been found. Delete from wierd list.\n")
    
for i in range(0, len(data_length_secc_send_split)):
#for i in [7]:
    print("\n-------- Test Round " + str(i+1) + " --------")
    #Stage 1
    secc_send_time_anchor_each_round = [timelist_secc_send_split[i][0]]
    stageNum = 1
    printinfo(stageNum, frame_num_secc_send_split[i][0], info_secc_send_split[i][0])
    stageNum = stageNum + 1
    
    #Deal with packet loss......
    if(i == 1 and packet_loss_testing_round_1_flag == False):
        secc_send_time_anchor_each_round.append(47.83142)
        printinfo(stageNum, 220571, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        packet_loss_testing_round_1_flag = True
        
    #Deal with packet loss......
    if(i == 4 and packet_loss_testing_round_4_flag == False):
        secc_send_time_anchor_each_round.append(114.408053)
        printinfo(stageNum, 562244, "Packet Loss")
        print_time_difference(secc_send_time_anchor_each_round)
        stageNum = stageNum + 1
        packet_loss_testing_round_4_flag = True
    
    
    for j in range(1, len(data_length_secc_send_split[i])):
        #Stage 2
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 2, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 3
        if(data_length_secc_send_split[i][j] == 157 and data_length_secc_send_split[i][j-1] != 157):
            if(checkstage_num(stageNum, 3, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 4
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 4, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                    
                continue
                
        #Stage 5
        if(data_length_secc_send_split[i][j] == 253 and data_length_secc_send_split[i][j-1] != 253):
            if(checkstage_num(stageNum, 5, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 6
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 6, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 7
        if(data_length_secc_send_split[i][j] == 189 and data_length_secc_send_split[i][j-1] != 189):
            if(checkstage_num(stageNum, 7, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                #Deal with packet loss......
                if(i == 7 and packet_loss_testing_round_7_flag == False):
                    secc_send_time_anchor_each_round.append(181.844227)
                    printinfo(stageNum, 926419, "Packet Loss")
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    packet_loss_testing_round_7_flag = True
                
                continue
                
        #Stage 8
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 8, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        #Stage 9
        if(data_length_secc_send_split[i][j] == 493 and data_length_secc_send_split[i][j-1] != 493):
            if(checkstage_num(stageNum, 9, info_secc_send_split[i][j],
                                  timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                  seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                print_time_difference(secc_send_time_anchor_each_round)
                stageNum = stageNum + 1
                
                continue
                
        #Stage 10
        if(data_length_secc_send_split[i][j] == 173 and data_length_secc_send_split[i][j-1] != 173):
            if(info_secc_send_split[i][j][-2:] == 'TC'):
                if(checkstage_num(stageNum, 10, info_secc_send_split[i][j],
                                      timelist_secc_send_split[i][j], data_length_secc_send_split[i][j],
                                      seq_num_secc_send_split[i][j], frame_num_secc_send_split[i][j])):
                    secc_send_time_anchor_each_round.append(timelist_secc_send_split[i][j])
                    printinfo(stageNum, frame_num_secc_send_split[i][j], info_secc_send_split[i][j])
                    print_time_difference(secc_send_time_anchor_each_round)
                    stageNum = stageNum + 1
                    
                    continue
                    
            else:
                print("Stage " + str(stageNum) +
                      ", Not TC? No. " + str(frame_num_secc_send_split[i][j]) +
                      ", Data length " + str(data_length_secc_send_split[i][j]) +
                      ", Info " + str(info_secc_send_split[i][j]))
        
        if(stageNum == 11): break
        
    secc_send_time_anchor.append(secc_send_time_anchor_each_round)


-------- Test Round 1 --------
-----------------------------------------
Stage 1 finish. No. 0, Info QoS Data, SN=2652, FN=0, Flags=.p.....TC

What is this packet? Data Length = 189 No. 96150
Time 26.56558 #Seq 2614 Info QoS Data, SN=2614, FN=0, Flags=.p.....TC


What is this packet? Data Length = 189 No. 96150
Time 26.56558 #Seq 2614 Info QoS Data, SN=2614, FN=0, Flags=.p.....TC

-----------------------------------------
Stage 2 finish. No. 96282, Info QoS Data, SN=2618, FN=0, Flags=.p.....TC
Time difference: 6.713229999999999

What is this packet? Data Length = 189 No. 97390
Time 26.987199 #Seq 2620 Info QoS Data, SN=2620, FN=0, Flags=.p.....TC


What is this packet? Data Length = 189 No. 97390
Time 26.987199 #Seq 2620 Info QoS Data, SN=2620, FN=0, Flags=.p.....TC


What is this packet? Data Length = 173 No. 97619
Time 27.144891 #Seq 2622 Info QoS Data, SN=2622, FN=0, Flags=.p.....TC


What is this packet? Data Length = 173 No. 97619
Time 27.144891 #Seq 2622 Info QoS Data, SN=2622, 

In [20]:
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(str(i) + ": " + str(len(secc_send_time_anchor[i])))

20

0: 3
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 10
9: 10
10: 10
11: 10
12: 10
13: 10
14: 10
15: 4
16: 10
17: 10
18: 10
19: 10


In [21]:
#for debug
for i in range(0, len(secc_send_time_anchor)):
    print(secc_send_time_anchor[i][len(secc_send_time_anchor[i])-1]-secc_send_time_anchor[i][0])

24.109099
2.772869
2.8216469999999987
2.2352659999999958
3.0744860000000074
2.701705000000004
1.9751569999999958
3.305946000000006
2.595867999999996
2.9405849999999987
2.738280000000003
1.9751799999999662
2.2479170000000295
1.3351150000000302
2.1437290000000075
1.7634969999999726
2.4340329999999994
2.2880719999999997
1.9957169999999564
3.167607999999973


In [22]:
#BLOCK ACK processing
#Time evcc sends blockACK after secc send
evcc_blockAck_time_anchor = []

blockack_index = 0
for i in range(0, len(secc_send_time_anchor)):
    evcc_blockAck_time_anchor_in_each_round = []
    
    for j in range(0, len(secc_send_time_anchor[i])):
        while(timelist_block_ack[blockack_index] < secc_send_time_anchor[i][j]):
            blockack_index = blockack_index + 1
        evcc_blockAck_time_anchor_in_each_round.append(timelist_block_ack[blockack_index])
        #print("ACK time difference = " + str(
        #timelist_block_ack[blockack_index] - secc_send_time_anchor[i][j]))
    
    evcc_blockAck_time_anchor.append(evcc_blockAck_time_anchor_in_each_round)

In [23]:
#SECC Packet Loss dict & Block Ack Loss
#secc_packet_loss_dict[i] = {stageNum}
# i = 5, j = 0 is packet loss of evcc BLOCK ACK
# i = 3, j = 9 is packet loss of evcc BLOCK ACK
secc_packet_loss_dict = {}
secc_packet_loss_dict[2] = {5}
secc_packet_loss_dict[1] = {1, 3, 4, 5}
secc_packet_loss_dict[4] = {1}
secc_packet_loss_dict[7] = {7}


#for i in secc_packet_loss_dict.get(5): print(i)
#7 in secc_packet_loss_dict.get(5)

In [24]:
#Calculate the avg. block ack latency
sum_latency = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) == None):
            sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
            cnt = cnt + 1
        elif(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)): continue
            else:
                sum_latency = sum_latency + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency = sum_latency/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency) + " s")
print(cnt)

Avg Block Ack latency = 0.08593851111111366 s
180


In [25]:
#For curiousity
#Calculate the avg. block ack latency of the appending packet loss packets
sum_latency_2 = 0
cnt = 0
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                sum_latency_2 = sum_latency_2 + (evcc_blockAck_time_anchor[i][j]-secc_send_time_anchor[i][j])
                cnt = cnt + 1
        else: continue
avg_blockAck_latency_2 = sum_latency_2/cnt
print("Avg Block Ack latency = " + str(avg_blockAck_latency_2) + " s")
print(cnt)

Avg Block Ack latency = 0.1844141428571433 s
7


In [26]:
#For debug. Can see testing round 6 (i = 5) has longer block ack latency due to packet loss.
# I only append the secc send time, but didn't append the block time.
from statistics import mean

for i in range(0, len(evcc_blockAck_time_anchor)):
    print("i = " + str(i) + ": " + str(mean(evcc_blockAck_time_anchor[i])-mean(secc_send_time_anchor[i])))

i = 0: 1.7933803333333316
i = 1: 0.1131196999999986
i = 2: 0.1325743000000017
i = 3: 0.07842660000000024
i = 4: 0.06599009999999339
i = 5: 0.048772299999995994
i = 6: 0.034205700000001116
i = 7: 0.09362169999999992
i = 8: 0.08498069999998847
i = 9: 0.07000110000001314
i = 10: 0.06513179999998897
i = 11: 0.04307069999998703
i = 12: 0.036978099999998904
i = 13: 0.009402000000022781
i = 14: 0.03518860000002633
i = 15: 0.05435749999998052
i = 16: 0.03496380000001409
i = 17: 0.049999899999988884
i = 18: 0.03877399999998943
i = 19: 0.08102489999998852


In [27]:
#Use the avg. block ack latency for the lost secc sent packets
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_packet_loss_dict.get(i) != None):
            if(j in secc_packet_loss_dict.get(i)):
                evcc_blockAck_time_anchor[i][j] = secc_send_time_anchor[i][j] + avg_blockAck_latency
                print("i = " + str(i))
                print("j = " + str(j))
        else: continue

i = 1
j = 1
i = 1
j = 3
i = 1
j = 4
i = 1
j = 5
i = 2
j = 5
i = 4
j = 1
i = 7
j = 7


In [28]:
#Test Round 1, 16: evcc send Too many lost packets... but no timeout

for i in [15, 0]:
    del evcc_send_time_anchor[i]
    del secc_send_time_anchor[i]
    del evcc_blockAck_time_anchor[i]


#for debug
print(str(len(evcc_send_time_anchor)) + '\n')
for i in range(0, len(evcc_send_time_anchor)):
    print(len(evcc_send_time_anchor[i]))
print('\n')    
    
#for debug
print(str(len(secc_send_time_anchor)) + '\n')
for i in range(0, len(secc_send_time_anchor)):
    print(len(secc_send_time_anchor[i]))
print('\n')
    
#for debug
print(str(len(evcc_blockAck_time_anchor)) + '\n')
for i in range(0, len(evcc_blockAck_time_anchor)):
    print(len(evcc_blockAck_time_anchor[i]))

18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


18

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [29]:
#for debug check wierd value
for i in range(0, len(evcc_blockAck_time_anchor)):
    for j in range(0, len(evcc_blockAck_time_anchor[i])):
        if(secc_send_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))
        if(evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j] < 0):
            print("i = " + str(i) + " j = " + str(j))

In [30]:
#For debug
i = 1
for j in [5]:
    print("Stage " + str(j+1))
    print(evcc_send_time_anchor[i][j])
    print(secc_send_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j])
    print(evcc_blockAck_time_anchor[i][j]-evcc_send_time_anchor[i][j])

Stage 6
70.490289
70.499881
70.58581951111111
0.09553051111110733


In [313]:
path = 'time_record/max_interf_80211.txt'
f = open(path, 'w')
for i in range(0, len(evcc_send_time_anchor)):
    for j in range(0, len(evcc_send_time_anchor[i])):
        print((evcc_blockAck_time_anchor[i][j] - evcc_send_time_anchor[i][j])*1000, file=f, end="")
        f.writelines('\t')
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    print((evcc_blockAck_time_anchor[i][len(evcc_blockAck_time_anchor[i])-1] - evcc_send_time_anchor[i][0])*1000, file=f, end="")
    f.writelines('\t')
    f.writelines('20')
    f.writelines('\t')
    f.writelines('0')
    f.writelines('\t')
    f.writelines('\n')
f.close()

In [314]:
#如果 TCP 有 retransmission，在 Data Link Layer seq 會往下傳送，很難抓取